<a href="https://colab.research.google.com/github/ronyavivi/PhotoBombFix/blob/main/Full_pipeline_with_site.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Credits : 
https://colab.research.google.com/github/tensorflow/tpu/blob/master/models/official/mask_rcnn/mask_rcnn_demo.ipynb#scrollTo=DL_0tSC67biu

https://github.com/tensorflow/models/blob/master/research/object_detection/utils/visualization_utils.py

https://advimman.github.io/lama-project/

Website link : https://3e6u7yt7fdew4msm.anvil.app/4AKYTHGA6YHKTACKEAUX5KF3

##About :

This is the final notebook tha works with the site. due to gpu the second part doen't works with the site. 

You can get your prediction by going to : "Manually run the final step" box, enter the bomber number and run the next 2 cells. 

<br> In order to run the notebook in another drive change this path wherever you see it :
<br> '/content/gdrive/My Drive/Colab_Notebooks/Final_project'
<br> you will find all the locations to change by pressing ctrl+f and search "to_change".
<br><br> **The notebook will work only with sufficient GPU. **

##About the site 


*   First part of segmentation works and you can use it :)
*   Second part of inpainting only work after running the code manually and adding the inpainted image to the output folder.



## Download the source code
Download the source code of the Mask R-CNN model.

In [ ]:
 !git clone https://github.com/tensorflow/tpu/

In [ ]:
!git clone https://github.com/tensorflow/models/tree/master/research/object_detection

## Import libraries

In [ ]:
!git clone https://github.com/tensorflow/models/blob/master/research/object_detection/utils/visualization_utils.py
from IPython import display
from PIL import Image
import numpy as np
import tensorflow as tf
import sys
sys.path.insert(0, 'tpu/models/official')
sys.path.insert(0, 'tpu/models/official/mask_rcnn')
import coco_metric
import collections
import six
from mask_rcnn.object_detection import visualization_utils
from mask_rcnn.object_detection import shape_utils
import cv2
import base64
import numpy as np
import os
from shutil import copyfile
import shutil

In [ ]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import sys 
sys.path.insert(0,'/content/gdrive/My Drive/Colab_Notebooks/Final_project')##to_change

In [ ]:
os.chdir('/content/gdrive/MyDrive/Colab_Notebooks/Final_project') ##to_change

In [ ]:
import visualization_utils_adjusted

## Load the COCO index mapping
This Colab uses a pretrained checkpoint of the Mask R-CNN model that is trained using the COCO dataset. Here is the mapping between the indices that the model predicts and the categories in text.

In [ ]:
ID_MAPPING = {
    1: 'person',
    2: 'bicycle',
    3: 'car',
    4: 'motorcycle',
    5: 'airplane',
    6: 'bus',
    7: 'train',
    8: 'truck',
    9: 'boat',
    10: 'traffic light',
    11: 'fire hydrant',
    13: 'stop sign',
    14: 'parking meter',
    15: 'bench',
    16: 'bird',
    17: 'cat',
    18: 'dog',
    19: 'horse',
    20: 'sheep',
    21: 'cow',
    22: 'elephant',
    23: 'bear',
    24: 'zebra',
    25: 'giraffe',
    27: 'backpack',
    28: 'umbrella',
    31: 'handbag',
    32: 'tie',
    33: 'suitcase',
    34: 'frisbee',
    35: 'skis',
    36: 'snowboard',
    37: 'sports ball',
    38: 'kite',
    39: 'baseball bat',
    40: 'baseball glove',
    41: 'skateboard',
    42: 'surfboard',
    43: 'tennis racket',
    44: 'bottle',
    46: 'wine glass',
    47: 'cup',
    48: 'fork',
    49: 'knife',
    50: 'spoon',
    51: 'bowl',
    52: 'banana',
    53: 'apple',
    54: 'sandwich',
    55: 'orange',
    56: 'broccoli',
    57: 'carrot',
    58: 'hot dog',
    59: 'pizza',
    60: 'donut',
    61: 'cake',
    62: 'chair',
    63: 'couch',
    64: 'potted plant',
    65: 'bed',
    67: 'dining table',
    70: 'toilet',
    72: 'tv',
    73: 'laptop',
    74: 'mouse',
    75: 'remote',
    76: 'keyboard',
    77: 'cell phone',
    78: 'microwave',
    79: 'oven',
    80: 'toaster',
    81: 'sink',
    82: 'refrigerator',
    84: 'book',
    85: 'clock',
    86: 'vase',
    87: 'scissors',
    88: 'teddy bear',
    89: 'hair drier',
    90: 'toothbrush',
}
category_index = {k: {'id': k, 'name': ID_MAPPING[k]} for k in ID_MAPPING}

## Load an image
Now, you can load an image. Use either the sample image included here, or update the field with an image of your choice.

## Create a Tensorflow session
Now let us create a Tensorflow session to run the inference. You can either connect to a TPU or a normal CPU backend.

In [ ]:
!pip install anvil-uplink
import anvil.server

anvil.server.connect("server_2BAUIPXF2NECUCB2PLF6DJF4-3E6U7YT7FDEW4MSM")

# Prepare a full pipeline function to connect to the server

In [ ]:
##function 1 - get image
import io
from PIL import Image

@anvil.server.callable
def get_image(img):
  os.chdir('/content/gdrive/MyDrive/Colab_Notebooks/Final_project') ##to_change
  shutil.rmtree('./data_for_prediction', ignore_errors=True)
  os.mkdir('./data_for_prediction')
  img = Image.open(io.BytesIO(img.get_bytes()))
  img.save('./data_for_prediction/test.jpg')
  return 


In [ ]:
##function 2 - get image segmentation
import json 
import pickle

@anvil.server.callable
def get_segmentation() :
  os.chdir('/content/gdrive/MyDrive/Colab_Notebooks/Final_project') ##to_change
  session = tf.compat.v1.Session(graph=tf.Graph())  
  #open image
  image_path = '/content/gdrive/MyDrive/Colab_Notebooks/Final_project/data_for_prediction/test.jpg'##to_change
  with open(image_path, 'rb') as f:
    np_image_string = np.array([f.read()])
  image = Image.open(image_path)
  width, height = image.size
  h = height
  file = open('height_file.txt', 'wb')
  pickle.dump(h, file)
  file.close()
  w = width
  file = open('width_file.txt', 'wb')
  pickle.dump(w, file)
  file.close()

  #get model
  np_image = np.array(image.getdata()).reshape(height, width, 3).astype(np.uint8)
  saved_model_dir = 'gs://cloud-tpu-checkpoints/mask-rcnn/1555659850' 
  _ = tf.compat.v1.saved_model.load(session, ['serve'], saved_model_dir)
  feed_dict = {}

  #get prediction
  global segmentations
  global detection_boxes
  with session as session:
    num_detections, detection_boxes, detection_classes, detection_scores, detection_masks, image_info = session.run(
        ['NumDetections:0', 'DetectionBoxes:0', 'DetectionClasses:0', 'DetectionScores:0', 'DetectionMasks:0', 'ImageInfo:0'],
        feed_dict={'Placeholder:0': np_image_string})
    
  #edit prediction
  detection_classes=detection_classes.astype(np.int32)
  num = int(num_detections)
  to_be_removed = []
  for i in range(num) :
    if detection_classes[0][i] != 1:
      to_be_removed.append(i)
  to_be_removed.sort(reverse = True)
  for t in to_be_removed : 
    detection_classes = np.delete(detection_classes , t , axis = 1)
    detection_scores = np.delete(detection_scores , t , axis = 1)
    detection_boxes = np.delete(detection_boxes , t , axis = 1)
    detection_masks = np.delete(detection_masks , t , axis = 1)
  num_detections = (detection_classes.shape)[1]
  detection_boxes = np.squeeze(detection_boxes * image_info[0, 2], axis=(0,))[0:num_detections]
  detection_scores = np.squeeze(detection_scores, axis=(0,))[0:num_detections]
  detection_classes = np.squeeze(detection_classes.astype(np.int32), axis=(0,))[0:num_detections]
  instance_masks = np.squeeze(detection_masks, axis=(0,))[0:num_detections]
  ymin, xmin, ymax, xmax = np.split(detection_boxes, 4, axis=-1)
  processed_boxes = np.concatenate([xmin, ymin, xmax - xmin, ymax - ymin], axis=-1)
  segmentations = coco_metric.generate_segmentation_from_masks(instance_masks, processed_boxes, height, width)

  #Diaplay detections on image : 
  max_boxes_to_draw = 30 
  image_with_detections = visualization_utils_adjusted.visualize_boxes_and_labels_on_image_array(
      np_image,
      detection_boxes,
      detection_classes,
      detection_scores,
      category_index,
      instance_masks=segmentations,
      use_normalized_coordinates=False,
      max_boxes_to_draw=max_boxes_to_draw)

  image_with_detections = Image.fromarray(image_with_detections.astype(np.uint8))
  image_with_detections.save('/content/gdrive/MyDrive/Colab_Notebooks/Final_project/data_for_prediction/image_with_detections.png')##to_change
  b = io.BytesIO()
  image_with_detections.save(b, 'jpeg')
  im_bytes = b.getvalue()
  return anvil.BlobMedia("image/jpg",im_bytes)





## Perform instance segmentation and retrieve the predictions
Now let's run the inference and process the predictions from the model.


In [ ]:
def create_mask(height,width,segmentations,b):
    image_data = np.zeros((height, width), dtype=np.uint8)

    for row in range(segmentations[b].shape[0]): 
        for col in range(segmentations[b].shape[1]):
            if segmentations[b][row][col] == 1 : 
                image_data[row][col] = 255
                # Add pixels to the left of the mask
                if col >= 11:
                    image_data[row][col-11] = 255
                else:
                    image_data[row][:col] = 255
                # Add pixels to the right of the mask
                if col <= width-12:
                    image_data[row][col+11] = 255
                else:
                    image_data[row][col:] = 255
                # Add pixels to the top of the mask
                if row >= 11:
                    image_data[row-11][col] = 255
                else:
                    image_data[:row,col] = 255
                # Add pixels to the bottom of the mask
                if row <= height-12:
                    image_data[row+11][col] = 255
                else:
                    image_data[row:,col] = 255
    return image_data

In [ ]:
##function 3 - create mask
from IPython.display import display
bomber = 1

@anvil.server.callable
def create_cur_mask (bomber) : 
  print(os.getcwd())
  b = int(bomber) - 1
  file = open('height_file.txt', 'rb')
  height = pickle.load(file)
  file.close()
  file = open('width_file.txt', 'rb')
  width = pickle.load(file)
  file.close()
  os.remove('width_file.txt')
  os.remove('height_file.txt')
  cur_mask = create_mask(height, width ,segmentations,b)
  cur_mask = Image.fromarray(cur_mask)
  cur_mask.save('/content/gdrive/MyDrive/Colab_Notebooks/Final_project/data_for_prediction/test_mask.png')##to_change

  return ()


Add wait_forever for the server

In [ ]:
## imports for the second part : 
print('\n> Cloning the repo')
!git clone https://github.com/advimman/lama.git

print('\n> Install dependencies')
!pip install torch==1.8.0 torchvision==0.9.0 torchaudio==0.8.0 torchtext==0.9
!pip install -r lama/requirements.txt --quiet
!pip install wget --quiet
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html --quiet


print('\n> Changing the dir to:')
os.chdir('/content/gdrive/MyDrive/Colab_Notebooks/Final_project/lama')##to_change 

print('\n> Download the model')
!curl -L $(yadisk-direct https://disk.yandex.ru/d/ouP6l8VJ0HpMZg) -o big-lama.zip
!unzip big-lama.zip

print('>fixing opencv')
!pip uninstall opencv-python-headless -y --quiet
!pip install opencv-python-headless==4.1.2.30 --quiet


print('\n> Init mask-drawing code')
from google.colab.output import eval_js
from base64 import b64decode
from shutil import copyfile
# import shutil

In [ ]:
!pip install tensorflow-gpu==2.7.0+nv21.9 tensorflow-gpu-estimator==2.7.0
!pip install scikit-learn>=1.0.0


In [ ]:
## function 4 - erase and inpaint : 
import torch
torch.cuda.empty_cache()
@anvil.server.callable
def get_new_image():
  inpainted_path = '/content/gdrive/MyDrive/Colab_Notebooks/Final_project/output/test_mask.png'
  #inpainted_path = os.path.join('/content/output', 'test_mask.png')
  inpainted = np.array(Image.open(inpainted_path).convert('RGB'))
  inpainted = Image.fromarray(inpainted.astype(np.uint8))
  bb = io.BytesIO()
  inpainted.save(bb, 'jpeg')
  im_bytes = bb.getvalue()
  return anvil.BlobMedia("image/jpg",im_bytes)




###Manually run the final step

In [ ]:
b = #Add here the bomber num
create_cur_mask(b)

In [ ]:
os.chdir('/content/gdrive/MyDrive/Colab_Notebooks/Final_project/lama') ##to_change
shutil.rmtree('./data_for_prediction', ignore_errors=True)
os.mkdir('./data_for_prediction')
img_path = '/content/gdrive/MyDrive/Colab_Notebooks/Final_project/data_for_prediction/test.jpg'##to_change
img_name = os.path.basename(img_path)
copyfile(img_path, f'./data_for_prediction/{img_name}')
img = Image.open(img_path)
mask_path = '/content/gdrive/MyDrive/Colab_Notebooks/Final_project/data_for_prediction/test_mask.png'##to_change
mask = Image.open(mask_path)
mask_name = f'{os.path.splitext(img_name)[0]}_mask.png'
copyfile(mask_path, f'./data_for_prediction/{mask_name}')
img_name = os.path.splitext(img_name)[0]
mask_name = os.path.splitext(mask_name)[0]
img = np.array(img)[:,:,::-1] # Convert from RGB to BGR for OpenCV
mask = np.array(mask)
!PYTHONPATH=. TORCH_HOME=$(pwd) python3 bin/predict.py model.path=$(pwd)/big-lama indir=$(pwd)/data_for_prediction outdir=/content/output dataset.img_suffix=.jpg
shutil.rmtree('./data_for_prediction', ignore_errors=True)
inpainted_path = os.path.join('/content/output', 'test_mask.png')
inpainted = np.array(Image.open(inpainted_path).convert('RGB'))
inpainted = Image.fromarray(inpainted.astype(np.uint8))